In [ ]:
from packaging import version

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import tensorboard as tb

In [ ]:
experiment_id = "jsJp1dYFRnmrdo0ePWMwsg"
experiment_id = "90BjtXBNRBuLwwE5eHau7w"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
experiment

In [ ]:
df = experiment.get_scalars()

In [ ]:
print(df["run"].unique())
print(df["tag"].unique())

In [ ]:
dfw = df[df.run.str.endswith("100000.0/version_1") | df.run.str.endswith("100000.0/version_2")]
dfw = dfw.pivot_table(
        values="value",
        index=["run", "step"],
        columns="tag",
        dropna=False,
    )
dfw = dfw.reset_index()
dfw.columns.name = None
dfw.columns.names = [None for name in dfw.columns.names]
# dfw = dfw[['run', 'step', 'epoch', 'valid_loss', 'train_loss', 'valid_acc', 'grad_2.0_norm_total_step']]

In [ ]:
print(dfw["run"].unique())

In [ ]:
method_split = dfw.run.apply(lambda run: f"{run.split('-')[1]}".upper())
method_split.name = None

In [ ]:
plt.figure(figsize=(4.8, 4.8))
sns.set(font_scale=1.5, style="white")
sns.lineplot(data=dfw, x="epoch", y="valid_acc", style=method_split, color='black').set(title="Validation accuracy", ylabel="Accuracy (%)", xlabel="Trained epochs")
plt.tight_layout()
plt.savefig('hsd_accuracy.svg')

In [ ]:
exodus_df = dfw[dfw["run"] == "hsd/lightning_logs/hsd-exodus-100000.0/version_1"]
slayer_df = dfw[dfw["run"] == "hsd/lightning_logs/hsd-slayer-100000.0/version_1"]

gdf = pd.DataFrame({
    'algorithm': ['EXODUS', 'EXODUS', 'EXODUS', 'EXODUS', 'SLAYER', 'SLAYER', 'SLAYER', 'SLAYER'],
    'Layer': [1, 2, 3, 4, 1, 2, 3, 4],
    'Mean 2-norm': [
        exodus_df["grad_2.0_norm/network.0.0.weight_epoch"].mean(),
        exodus_df["grad_2.0_norm/network.1.0.weight_epoch"].mean(),
        exodus_df["grad_2.0_norm/network.2.0.weight_epoch"].mean(),
        exodus_df["grad_2.0_norm/network.3.weight_epoch"].mean(),
        slayer_df["grad_2.0_norm/linear_input.weight_epoch"].mean(),
        slayer_df["grad_2.0_norm/linear_hidden.0.weight_epoch"].mean(),
        slayer_df["grad_2.0_norm/linear_hidden.1.weight_epoch"].mean(),
        slayer_df["grad_2.0_norm/linear_output.weight_epoch"].mean(),
        ]
})
gdf

In [ ]:
sns.set(font_scale=0.9, style="white")
plt.figure(figsize=(4, 8))
g = sns.FacetGrid(gdf, col="algorithm", palette='flare')
g.map(plt.plot, "Layer", "Mean 2-norm", linestyle="-", marker="o")
g.set_titles("{col_name}")
plt.suptitle("Gradient norms")
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)
plt.yscale("log")
# for ax in g.axes.flatten():
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0, title="Surrogate scale")
plt.tight_layout()
# plt.subplots_adjust(top=0.8)
plt.savefig(f"hsd_mean_gradnorm.svg")